# LangChain Exercise Solutions
These reference notebooks `01` through `10` and rely on environment variables for API keys (e.g., `OPENAI_API_KEY`, `HUGGINGFACEHUB_API_TOKEN`).


## Exercise 1 – Prompt Template Warm‑Up


In [ ]:
from langchain_core.prompts import PromptTemplate

psa_template = PromptTemplate.from_template(
    """City of Dublin PSA:\nHey neighbors, let's talk about {topic}.\n"""
)

topic = "keeping community Wi-Fi safe"
formatted_psa = psa_template.format(topic=topic)
print(formatted_psa)


## Exercise 2 – LCEL Chain with ChatOpenAI


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import StrOutputParser

MODEL_NAME = "gpt-4o-mini"

topic = "urban community gardens"
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are an encouraging civic organizer who writes short calls-to-action."),
        ("human", "Create a 4-sentence update about {topic} that invites residents to participate."),
    ]
)

chain = prompt | ChatOpenAI(model=MODEL_NAME, temperature=0.2) | StrOutputParser()
print(chain.invoke({"topic": topic}))


## Exercise 3 – Routing with `RunnableBranch`


In [ ]:
from langchain_core.runnables import RunnableBranch

history_prompt = PromptTemplate.from_template(
    "You are a meticulous historian. Answer the question with dates and evidence.\nQuestion: {question}"
)
general_prompt = PromptTemplate.from_template(
    "You are a concise helpful assistant. Provide a practical answer to: {question}"
)

router = RunnableBranch(
    (lambda x: "history" in x["question"].lower(), history_prompt),
    general_prompt,
)

branch_chain = router | ChatOpenAI(model=MODEL_NAME, temperature=0.3) | StrOutputParser()
questions = [
    "How did Cold War history shape early space exploration?",
    "What are three ways to stay focused during remote work?",
]

for q in questions:
    answer = branch_chain.invoke({"question": q})
    print(f"Q: {q}\nA: {answer}\n")


## Exercise 4 – Retrieval on `kdot.txt`


In [ ]:
from langchain_community.document_loaders import TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

lyrics_loader = TextLoader("../data/kdot.txt", encoding="utf-8")
lyrics_docs = lyrics_loader.load()

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=80,
    add_start_index=True,
)
chunks = splitter.split_documents(lyrics_docs)

embeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(chunks, embeddings)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

rag_prompt = PromptTemplate.from_template(
    """Use the context to answer the question about Kendrick Lamar's recurring themes.\nContext:\n{context}\n\nQuestion: {question}\nHelpful answer (2 sentences):"""
)

rag_chain = (
    RunnableParallel({"context": retriever, "question": RunnablePassthrough()})
    | rag_prompt
    | ChatOpenAI(model=MODEL_NAME, temperature=0.2)
    | StrOutputParser()
)

question = "What does Kendrick say about resilience in the face of fear?"
print(rag_chain.invoke(question))
